In [1]:
import time
import numpy as np
import pandas as pd
import dwave
from dwave_qbsolv import QBSolv
from dwave.system import EmbeddingComposite, FixedEmbeddingComposite, DWaveSampler

In [2]:
# User defines upper triangular QUBO problem
def create_upper_matrix(values, size):
    upper = np.zeros((size, size))
    upper[np.triu_indices(size, 0)] = values
    return upper

In [3]:
def create_random_upper_matrix(n, density=0.05, max_aij=100):
    prob_zero = 1 - density
    num_values = 0
    for i in range(n):
        num_values += (i + 1)
    value_range = [x for x in range(-max_aij, max_aij)]
    value_probs = [prob_zero if x == 0 else (1 - prob_zero) / (len(value_range) - 1) for x in value_range]
    return create_upper_matrix(np.random.choice(value_range, p=value_probs, size=num_values), n)

In [4]:
def save_Q(Q, output_path):
    # note Q must be upper triangular for this
    strings = []
    num_edges = 0
    vertices = 0
    for edge, v in Q.items():
        if v != 0:
            strings.append(f'{edge[0]+1} {edge[1]+1} {-v}') #need to negate weights since MQLib is maximisation
            num_edges += 1
            vertices = max(vertices, edge[0]+1, edge[1]+1)
                
    strings.insert(0, f'{vertices} {num_edges}')
    with open(output_path, 'w') as f:
        f.writelines('\n'.join(strings))

In [5]:
def save_matrix(problem, output_path='data/instance.txt'):
    """Saves np matrix in MQLib format"""
    strings = []
    num_edges = 0
    for i in range(problem.shape[0]):
        for j in range(i, problem.shape[0]):
            if problem[i][j] != 0:
                strings.append(f'{i+1} {j+1} {-problem[i][j]}') #need to negate weights since MQLib is maximisation
                num_edges += 1
                
    strings.insert(0, f'{problem.shape[0]} {num_edges}')
    with open(output_path, 'w') as f:
        f.writelines('\n'.join(strings))

In [6]:
def load_matrix(input_path):
    with open(input_path, 'r') as f:
        lines = f.read().splitlines()
        n = int(lines[0].split()[0])
        problem = np.zeros((n, n))
        for line in lines[1:]:
            i, j, w = map(float, line.split())
            problem[int(i-1)][int(j-1)] = -w
        return problem

## Problem creation

In [20]:
# solution: y=-11, x=1001
#n = 4
#problem = create_upper_matrix([-5, 4, 8, 0, -3, 2, 0, -8, 10, -6], n)

In [11]:
n = 64
problem = create_random_upper_matrix(n, density=0.85)

In [18]:
Q = {}
for i in range(problem.shape[0]):
    for j in range(i, problem.shape[0]):
        if i == j:
            Q[(i, j)] = problem[i][j]
        else :
            Q[(i, j)] = problem[i][j] * 2

## Heuristic and Dwave

In [19]:
# Classical heuristic solver using tabu search
qbsolv = QBSolv()
start_time = time.time()
response = qbsolv.sample_qubo(Q, num_repeats=50)
print("--- %.3f seconds ---" % (time.time() - start_time))
samples = []
for sample in list(response.samples()):
    result = []
    for i in range(n):
        result.append(str(sample[i]))
    samples.append(''.join(result))
print("samples=" + str(samples))
print("energies=" + str(list(response.data_vectors['energy'])))

--- 0.224 seconds ---
samples=['0000100101101111100011000111001101101011111110011010101001011111', '1010010010001111011001100111110100100110111110010101111101010101', '1010000010001111011001100111110100100111111110010101111101010101', '1110010011001111010001000111010101101011111110110110111001010001', '1000100111101111110011000111111101100010111110010001101001010111']
energies=[-14199.0, -14089.0, -14081.0, -14044.0, -13936.0]


In [33]:
#embedding = dwave.embedding.pegasus.find_clique_embedding(100, 15)
#sampler = FixedEmbeddingComposite(DWaveSampler(), embedding=embedding)

In [90]:
# Define the sampler that will be used to run the problem
sampler = EmbeddingComposite(DWaveSampler())

# Run the problem on the sampler and print the results
start_time = time.time()
sampleset = sampler.sample_qubo(Q, num_reads=10, label=f'QUBO {n} variables')
print("--- %.3f seconds ---" % (time.time() - start_time))

print(sampleset)

--- 0.344 seconds ---
   0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 energy num_oc. chain_.
0  1  1  1  0  1  1  0  0  0  0  0  1  1  0  0  0  -54.0       1     0.0
1  1  1  0  1  1  1  1  0  0  0  0  0  1  0  0  0  -54.0       1     0.0
2  1  0  1  1  1  1  1  0  1  0  0  0  1  0  1  0  -54.0       1     0.0
3  0  0  1  0  1  1  1  1  0  0  0  0  1  1  0  0  -54.0       1     0.0
4  0  0  1  1  1  0  0  1  0  0  0  0  1  0  1  0  -54.0       1     0.0
5  0  1  0  0  1  1  1  0  0  0  0  0  1  1  0  0  -54.0       1     0.0
6  0  1  0  1  1  0  1  0  0  1  0  0  1  1  0  0  -54.0       1     0.0
7  1  1  1  1  1  0  1  0  1  0  0  1  1  0  1  0  -54.0       1     0.0
9  1  0  1  1  1  1  1  0  0  0  0  0  1  0  0  0  -54.0       1  0.0625
8  1  1  1  1  1  1  1  1  0  0  0  0  1  0  1  0  -49.0       1     0.0
['BINARY', 10 rows, 10 samples, 16 variables]


In [158]:
sampleset.to_pandas_dataframe()['energy'].mean()

-84.3

In [159]:
print(sampleset.info['timing']['qpu_access_time'] / 10**6)   

0.027374


## Random Target Graph

In [21]:
import dimod

In [22]:
target_structure = dimod.child_structure_dfs(DWaveSampler())

In [23]:
__, target_edgelist, target_adjacency = target_structure

In [24]:
len(target_adjacency)

5436

In [25]:
from collections import Counter
counter = Counter()
for i in range(160, 170):
    print(f'{i}: ', target_adjacency[i])
for k,v in target_adjacency.items():
    counter[len(v)] += 1

160:  {4800, 161, 4770, 4935, 4905, 4875, 4845, 175, 4815, 4785, 4920, 4890, 4860, 4830, 159}
161:  {160, 5025, 162, 4995, 4965, 5100, 5070, 176, 5040, 5010, 4980, 4950, 5115, 5085, 5055}
162:  {5280, 161, 5250, 5220, 5190, 5160, 5130, 5295, 177, 5265, 5235, 5175}
163:  {5475, 164, 5445, 5415, 5385, 5355, 5325, 178, 5460, 5430, 5400, 5370, 5340, 5310}
164:  {5505, 163, 5640, 5610, 5580, 5550, 5520, 5490, 179, 5655, 5625, 5595, 5565, 5535}
165:  {3105, 3075, 3045, 166, 3015, 2985, 3120, 3090, 3060, 150, 3030, 3000, 2970, 3135}
166:  {3255, 3300, 165, 3270, 167, 3240, 3210, 3180, 3150, 3315, 3285, 151, 3225, 3195, 3165}
167:  {3360, 3330, 3480, 166, 3495, 168, 3465, 3435, 3405, 3375, 3345, 152, 3450, 3420, 3390}
168:  {3585, 3555, 3525, 167, 169, 3660, 3630, 3600, 3570, 3540, 3510, 153, 3675, 3645, 3615}
169:  {3840, 3810, 3780, 3750, 168, 3720, 170, 3690, 3855, 3825, 3795, 3765, 3735, 3705, 154}


In [26]:
counter

Counter({5: 12,
         7: 162,
         6: 62,
         4: 1,
         10: 78,
         11: 175,
         9: 34,
         14: 1591,
         15: 2536,
         12: 165,
         13: 612,
         8: 5,
         3: 3})

In [246]:
# Problem generation
Q = {}
for edge in target_edgelist:
    Q[edge] = np.random.randint(-1, 2) # -1, 0, 1

In [247]:
#### Classical heuristic solver using tabu search
qbsolv = QBSolv()
start_time = time.time()
response = qbsolv.sample_qubo(Q, num_repeats=50)
print("--- %.3f seconds ---" % (time.time() - start_time))
print("energies=" + str(list(response.data_vectors['energy'])))

--- 45.854 seconds ---
energies=[-4761.0, -4758.0, -4755.0, -4755.0, -4754.0, -4754.0, -4753.0, -4752.0, -4752.0, -4751.0, -4750.0, -4750.0, -4750.0, -4748.0, -4748.0, -4747.0, -4747.0, -4746.0, -4745.0, -4745.0]


In [238]:
sampler = DWaveSampler()

# Run the problem on the sampler and print the results
start_time = time.time()
sampleset = sampler.sample_qubo(Q, num_reads=10, label=f'QUBO {n} variables')
print("--- %.3f seconds ---" % (time.time() - start_time))

print(sampleset)

--- 0.288 seconds ---
  30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 ... 5729  energy num_oc.
0  1  0  0  1  1  1  1  0  1  1  0  1  0  0  0  1  1 ...    1 -4742.0       1
1  0  1  0  1  1  1  1  0  1  1  0  1  1  0  0  1  1 ...    1 -4736.0       1
2  0  1  0  1  1  1  1  0  1  1  0  1  0  0  0  1  1 ...    1 -4734.0       1
3  0  1  0  1  1  1  1  0  1  1  0  1  0  0  0  1  1 ...    1 -4729.0       1
4  0  1  0  1  1  0  1  0  1  1  0  1  1  0  0  1  1 ...    1 -4728.0       1
5  0  1  0  1  1  1  1  0  1  1  0  1  0  0  0  1  1 ...    1 -4728.0       1
6  0  0  0  1  1  1  1  0  1  1  0  1  1  0  0  1  1 ...    1 -4726.0       1
7  1  0  0  1  1  1  1  0  1  1  0  1  0  0  0  1  1 ...    1 -4725.0       1
8  0  1  0  1  1  1  1  0  1  1  0  1  1  0  0  1  1 ...    1 -4721.0       1
9  0  1  0  1  1  1  1  0  1  1  0  1  1  0  0  1  1 ...    1 -4718.0       1
['BINARY', 10 rows, 10 samples, 5436 variables]


In [239]:
print(sampleset.info['timing']['qpu_access_time'] / 10**6)

0.027756


#### Qiskit

In [49]:
from qiskit.optimization.algorithms import CplexOptimizer, MinimumEigenOptimizer
from qiskit.aqua.algorithms import NumPyMinimumEigensolver
from qiskit.optimization.problems import QuadraticProgram

In [50]:
exact = MinimumEigenOptimizer(NumPyMinimumEigensolver()) # to solve QUBOs

# in case CPLEX is installed it can also be used for the convex problems, the QUBO,
# or as a benchmark for the full problem.
cplex = CplexOptimizer()

In [59]:
qubo = QuadraticProgram()
for i in range(n):
    qubo.binary_var(str(i))
quadratic = {(str(i), str(j)): problem[i][j] for i in range(n) for j in range(n)}
qubo.minimize(quadratic=quadratic)

In [96]:
cplex_result.fval

-373.0

In [60]:
start_time = time.time()
cplex_result = cplex.solve(qubo)
print("--- %.3f seconds ---" % (time.time() - start_time))
print(cplex_result)

--- 0.037 seconds ---
optimal function value: -373.0
optimal value: [1. 1. 0. 0. 1. 1. 0. 1. 0. 1. 1. 1. 0. 0. 0. 1.]
status: SUCCESS


In [ ]:
start_time = time.time()
exact_result = exact.solve(qubo)
print("--- %.3f seconds ---" % (time.time() - start_time))
print(exact_result)

## Generate QUBO of size n, evaluate on Tabu and D-Wave and repeat t times

In [26]:
def generate_and_evaluate_qubo(n=16, density=0.05, max_aij=100, num_shots=10, t=50, run_cplex=True, save_matrix=True):
    d = {'Tabu Min': [], 'Tabu Time': [], 'D-Wave Min': [], 'D-Wave Time (Total)': [], 'D-Wave Time (Quantum)': []}
    if run_cplex:
        d['CPLEX Min'] = []
        d['CPLEX Time'] = []
    
    for i in range(t):
        print(f'ITERATION {i}/{t}')
        problem = create_random_upper_matrix(n, density=density, max_aij=max_aij)
        if save_matrix:
            save_matrix(problem, f'data/random_qubo_instances/{n}_{int(density*100)}_{max_aij}_{i}.txt')

        Q = {}
        for i in range(problem.shape[0]):
            for j in range(i, problem.shape[0]):
                Q[(i, j)] = problem[i][j]

        # Classical heuristic solver using tabu search
        qbsolv = QBSolv()
        start_time = time.time()
        response = qbsolv.sample_qubo(Q, num_repeats=50)
        elapsed_time = time.time() - start_time
        min_energy = list(response.data_vectors['energy'])[0]
        d['Tabu Min'].append(min_energy)
        d['Tabu Time'].append(elapsed_time)

        # D-Wave
        try:
            sampler = EmbeddingComposite(DWaveSampler())
            start_time = time.time()
            sampleset = sampler.sample_qubo(Q, num_reads=num_shots, label=f'QUBO {n} variables')
            elapsed_time = time.time() - start_time
            d['D-Wave Min'].append(min(sampleset.to_pandas_dataframe()['energy']))
            d['D-Wave Time (Total)'].append(elapsed_time)
            d['D-Wave Time (Quantum)'].append(sampleset.info['timing']['qpu_access_time'] / 10**6)
        except ValueError: # no embedding found
            d['D-Wave Min'].append(np.NaN)
            d['D-Wave Time (Total)'].append(np.NaN)
            d['D-Wave Time (Quantum)'].append(np.NaN)
            

        # CPLEX
        if run_cplex:
            cplex = CplexOptimizer()
            qubo = QuadraticProgram()
            for i in range(n):
                qubo.binary_var(str(i))
            quadratic = {(str(i), str(j)): problem[i][j] for i in range(n) for j in range(n)}
            qubo.minimize(quadratic=quadratic)
            start_time = time.time()
            cplex_result = cplex.solve(qubo)
            elapsed_time = time.time() - start_time

            d['CPLEX Min'].append(cplex_result.fval)
            d['CPLEX Time'].append(elapsed_time)
        
        df = pd.DataFrame(data=d)
        df.to_csv(f'data/results/{n}n_{int(density*100)}p_{max_aij}aij_{num_shots}shots.csv')
    return df

In [136]:
results = generate_and_evaluate_qubo(n=128, density=0.85, t=50, run_cplex=False)

ITERATION 0/50
ITERATION 1/50
ITERATION 2/50
ITERATION 3/50
ITERATION 4/50
ITERATION 5/50
ITERATION 6/50
ITERATION 7/50
ITERATION 8/50
ITERATION 9/50
ITERATION 10/50
ITERATION 11/50
ITERATION 12/50
ITERATION 13/50
ITERATION 14/50
ITERATION 15/50
ITERATION 16/50
ITERATION 17/50
ITERATION 18/50
ITERATION 19/50
ITERATION 20/50
ITERATION 21/50
ITERATION 22/50
ITERATION 23/50
ITERATION 24/50
ITERATION 25/50
ITERATION 26/50
ITERATION 27/50
ITERATION 28/50
ITERATION 29/50
ITERATION 30/50
ITERATION 31/50
ITERATION 32/50
ITERATION 33/50
ITERATION 34/50
ITERATION 35/50
ITERATION 36/50
ITERATION 37/50
ITERATION 38/50
ITERATION 39/50
ITERATION 40/50
ITERATION 41/50
ITERATION 42/50
ITERATION 43/50
ITERATION 44/50
ITERATION 45/50
ITERATION 46/50
ITERATION 47/50
ITERATION 48/50
ITERATION 49/50


In [137]:
results.mean()

Tabu Min                -24289.420000
Tabu Time                    0.523315
D-Wave Min              -19759.400000
D-Wave Time (Total)        282.690792
D-Wave Time (Quantum)        0.027665
dtype: float64

## Native graph

In [8]:
import dimod
target_structure = dimod.child_structure_dfs(DWaveSampler())
__, target_edgelist, target_adjacency = target_structure

In [ ]:
def generate_and_evaluate_native_qubo(max_aij=100, density=0.85, t=50):
    d = {'Tabu Min': [], 'Tabu Time': [], 'D-Wave Min': [], 'D-Wave Time (Total)': [], 'D-Wave Time (Quantum)': []}
    
    value_range = [x for x in range(-max_aij, max_aij)]
    value_probs = [1 - density if x == 0 else (density) / (len(value_range) - 1) for x in value_range]
    
    for i in range(t):
        print(f'ITERATION {i}/{t}')
        Q = {}
        for edge in target_edgelist:
            Q[edge] = np.random.choice(value_range, p=value_probs)
            
        save_Q(Q, f'data/random_qubo_instances/5730_{int(density*100)}_{i}.txt')

        # Classical heuristic solver using tabu search
        qbsolv = QBSolv()
        start_time = time.time()
        response = qbsolv.sample_qubo(Q, num_repeats=50)
        elapsed_time = time.time() - start_time
        min_energy = list(response.data_vectors['energy'])[0]
        d['Tabu Min'].append(min_energy)
        d['Tabu Time'].append(elapsed_time)

        # D-Wave
        try:
            sampler = DWaveSampler()
            start_time = time.time()
            sampleset = sampler.sample_qubo(Q, num_reads=10, label=f'QUBO 5730 variables')
            elapsed_time = time.time() - start_time
            d['D-Wave Min'].append(min(sampleset.to_pandas_dataframe()['energy']))
            d['D-Wave Time (Total)'].append(elapsed_time)
            d['D-Wave Time (Quantum)'].append(sampleset.info['timing']['qpu_access_time'] / 10**6)
        except ValueError: # no embedding found
            d['D-Wave Min'].append(np.NaN)
            d['D-Wave Time (Total)'].append(np.NaN)
            d['D-Wave Time (Quantum)'].append(np.NaN)
        
        df = pd.DataFrame(data=d)
        df.to_csv(f'data/results/5730_{int(density*100)}.csv')
    return df

In [ ]:
generate_and_evaluate_native_qubo(max_aij=100, density=0.85, t=50)

## Maximum Tabu can reach

In [52]:
n = 20000
problem = create_random_upper_matrix(n, density=0.85)
Q = {}
for i in range(problem.shape[0]):
    for j in range(i, problem.shape[0]):
        Q[(i, j)] = problem[i][j]

# Classical heuristic solver using tabu search
qbsolv = QBSolv()
start_time = time.time()
response = qbsolv.sample_qubo(Q, num_repeats=10)
elapsed_time = time.time() - start_time
min_energy = list(response.data_vectors['energy'])[0]
print(min_energy)
print(elapsed_time)

-95406584.0
2959.8904869556427


## D-Wave Accuracy tests

### Max Aij

In [ ]:
df = generate_and_evaluate_qubo(n=128, density=0.85, max_aij=1, t=10, run_cplex=False, save_matrix=False)

In [40]:
df.mean()

Tabu Min                -3947.500000
Tabu Time                   0.361288
D-Wave Min              -3899.600000
D-Wave Time (Total)       321.383720
D-Wave Time (Quantum)       0.027636
dtype: float64

### Number of shots

In [48]:
df = generate_and_evaluate_qubo(n=128, density=0.85, max_aij=100, num_shots=1000, t=10, run_cplex=False, save_matrix=False)

ITERATION 0/10
ITERATION 1/10
ITERATION 2/10
ITERATION 3/10
ITERATION 4/10
ITERATION 5/10
ITERATION 6/10
ITERATION 7/10
ITERATION 8/10
ITERATION 9/10


In [49]:
df.mean()

Tabu Min                -24335.900000
Tabu Time                    0.460835
D-Wave Min              -22048.900000
D-Wave Time (Total)        309.475116
D-Wave Time (Quantum)        0.190572
dtype: float64

In [50]:
-22048.9 / -24335.9

0.9060236112081328